In [1]:
import pandas as pd
import pickle
import os
from vehicle import Vehicle

pickle_path = 'data/inference/AA_rdbt/simulation_initialization/initial_clips/01/02/2021-05-02 10-03-48-671927.pickle'

with open(pickle_path, 'rb') as f:
    vehicle_list = pickle.load(f)
    
    # 现在vehicle_list包含了pickle文件中存储的Vehicle对象列表
    print(vehicle_list)
    print(len(vehicle_list))
    print(vehicle_list[0].location.x)
    print(vehicle_list[0].location.y)
    print(vehicle_list[0].size.width) 
    print(vehicle_list[0].size.height)
    print(vehicle_list[0].size.length)
    print(vehicle_list[0].safe_size.width) 
    print(vehicle_list[0].safe_size.height)
    print(vehicle_list[0].safe_size.length)
    print(vehicle_list[0].id) 
    print(vehicle_list[0].speed_heading)
    print(vehicle_list[0].poly_box)
    print(vehicle_list[0].realworld_4_vertices)
    print(vehicle_list[0].safe_poly_box)


[<vehicle.vehicle.Vehicle object at 0x7f2bd2ec1550>, <vehicle.vehicle.Vehicle object at 0x7f2b3d962700>, <vehicle.vehicle.Vehicle object at 0x7f2b3d4aa5e0>]
3
86.76476951608201
-25.098855800926685
1.8
1.5
3.6
2.0
1.5
3.8000000000000003
1
163.5021444664423
POLYGON ((84.78329334828082 -25.45064010477762, 85.29445637232055 -23.724745448996288, 88.7462456838832 -24.74707149707575, 88.23508265984347 -26.472966152857083, 84.78329334828082 -25.45064010477762))
[[ 84.78329335 -25.4506401 ]
 [ 85.29445637 -23.72474545]
 [ 88.74624568 -24.7470715 ]
 [ 88.23508266 -26.47296615]]
POLYGON ((84.65901236606854 -25.51812519542993, 85.22697128166824 -23.600464466784008, 88.87052666609549 -24.67958640642344, 88.30256775049578 -26.597247135069363, 84.65901236606854 -25.51812519542993))


/tmp/ipykernel_742526/2170161046.py:9: UserWarning: Unpickling a shapely <2.0 geometry object. Please save the pickle again; shapely 2.1 will not have this compatibility.
  vehicle_list = pickle.load(f)


In [ ]:
import pandas as pd
import pickle
import os
import glob
from tqdm import tqdm
import logging
import cv2
import numpy as np
from vehicle import Vehicle
from geo_engine import GeoEngine

base_map_dir = 'data_yz/inference/rcu_252/basemap/252.png'
initail_region_dir = 'data_yz/inference/rcu_252/ROIs-map/vehicle-initial-region'
initial_pickle_dir = 'data_yz/inference/rcu_252/simulation_initialization/initial_clips'
output_initial_vehicle_filename = 'data_yz/inference/rcu_252/simulation_initialization/gen_veh_states/initial_vehicle_dict_0403.pickle'

map_width = 1896
map_height = 2091

if initail_region_dir is not None:
    circle_1_in1_map = cv2.imread(os.path.join(initail_region_dir, '1_in1.png'), cv2.IMREAD_GRAYSCALE)
    circle_1_in2_map = cv2.imread(os.path.join(initail_region_dir, '1_in2.png'), cv2.IMREAD_GRAYSCALE)
    circle_1_in3_map = cv2.imread(os.path.join(initail_region_dir, '1_in3.png'), cv2.IMREAD_GRAYSCALE)

    circle_2_in1_map = cv2.imread(os.path.join(initail_region_dir, '2_in1.png'), cv2.IMREAD_GRAYSCALE)
    circle_2_in2_map = cv2.imread(os.path.join(initail_region_dir, '2_in2.png'), cv2.IMREAD_GRAYSCALE)
    circle_2_in3_map = cv2.imread(os.path.join(initail_region_dir, '2_in3.png'), cv2.IMREAD_GRAYSCALE)
    circle_2_in4_map = cv2.imread(os.path.join(initail_region_dir, '2_in4.png'), cv2.IMREAD_GRAYSCALE)

    circle_3_in1_map = cv2.imread(os.path.join(initail_region_dir, '3_in1.png'), cv2.IMREAD_GRAYSCALE)
    circle_3_in2_map = cv2.imread(os.path.join(initail_region_dir, '3_in2.png'), cv2.IMREAD_GRAYSCALE)
    circle_3_in3_map = cv2.imread(os.path.join(initail_region_dir, '3_in3.png'), cv2.IMREAD_GRAYSCALE)
    
    circle_1_in1_map = cv2.resize(circle_1_in1_map, (map_width, map_height))
    circle_1_in2_map = cv2.resize(circle_1_in2_map, (map_width, map_height))
    circle_1_in3_map = cv2.resize(circle_1_in3_map, (map_width, map_height))

    circle_2_in1_map = cv2.resize(circle_2_in1_map, (map_width, map_height))
    circle_2_in2_map = cv2.resize(circle_2_in2_map, (map_width, map_height))
    circle_2_in3_map = cv2.resize(circle_2_in3_map, (map_width, map_height))
    circle_2_in4_map = cv2.resize(circle_2_in4_map, (map_width, map_height))

    circle_3_in1_map = cv2.resize(circle_3_in1_map, (map_width, map_height))
    circle_3_in2_map = cv2.resize(circle_3_in2_map, (map_width, map_height))
    circle_3_in3_map = cv2.resize(circle_3_in3_map, (map_width, map_height))

def initial_region_position_matching(pxl_pt):
    initial_region_position = 'not_initial_region'
    y0, x0 = pxl_pt[0], pxl_pt[1]

    # circle_1 entrance
    if circle_1_in1_map[x0, y0] > 128.:
        initial_region_position = '1_in1'
        return initial_region_position
    if circle_1_in2_map[x0, y0] > 128.:
        initial_region_position = '1_in2'
        return initial_region_position
    if circle_1_in3_map[x0, y0] > 128.:
        initial_region_position = '1_in3'
        return initial_region_position
    
    # circle_2 entrance
    if circle_2_in1_map[x0, y0] > 128.:
        initial_region_position = '2_in1'
        return initial_region_position
    if circle_2_in2_map[x0, y0] > 128.:
        initial_region_position = '2_in2'
        return initial_region_position
    if circle_2_in3_map[x0, y0] > 128.:
        initial_region_position = '2_in3'
        return initial_region_position
    if circle_2_in4_map[x0, y0] > 128.:
        initial_region_position = '2_in4'
        return initial_region_position
    
    # circle_1 entrance
    if circle_3_in1_map[x0, y0] > 128.:
        initial_region_position = '3_in1'
        return initial_region_position
    if circle_3_in2_map[x0, y0] > 128.:
        initial_region_position = '3_in2'
        return initial_region_position
    if circle_3_in3_map[x0, y0] > 128.:
        initial_region_position = '3_in3'
        return initial_region_position
    
base_map = GeoEngine(base_map_dir, map_height, map_width)

initial_vehicle_set = {}
initial_vehicle_set['1_in1'] = []
initial_vehicle_set['1_in2'] = []
initial_vehicle_set['1_in3'] = []
initial_vehicle_set['2_in1'] = []
initial_vehicle_set['2_in2'] = []
initial_vehicle_set['2_in3'] = []
initial_vehicle_set['2_in4'] = []
initial_vehicle_set['3_in1'] = []
initial_vehicle_set['3_in2'] = []
initial_vehicle_set['3_in3'] = []

logging.basicConfig(filename='252_initial_gen_0403.log', level=logging.INFO)

path_to_traj_data = initial_pickle_dir
subfolders = sorted(os.listdir(os.path.join(path_to_traj_data))) # 001
subsubfolders = [sorted(os.listdir(os.path.join(path_to_traj_data, subfolders[i]))) for i in
                    range(len(subfolders))] # 以rcu_252_0306为例，002~007
# print(len(subfolders))
# print(subfolders)
# print(len(subsubfolders[0])) # 
# print(subsubfolders[0][1]) #

for i in range(len(subfolders)):
    for j in tqdm(range(len(subsubfolders[i])), desc='searching initial regions'): # len(subsubfolders[i])
        # print(j)
        files_list = sorted(glob.glob(os.path.join(path_to_traj_data, subfolders[i], subsubfolders[i][j], '*.pickle')))
        buff_name = subsubfolders[i][j]
        # print(f'start generating features of {buff_name}')
        pickle_sum = len(files_list)
        try:
            for k in tqdm(range(pickle_sum),desc='getting initial region vehicles of subsubfolder'):
                vehicle_list = pickle.load(open(files_list[k], "rb")) # file_list[i]是一个pickle，即一帧
                for v in vehicle_list:
                    pxl_pt = base_map._world2pxl([v.location.x, v.location.y])
                    pxl_pt[1] = np.clip(pxl_pt[1], a_min=0, a_max=map_width-1) # road_map (map_width, map_height)
                    pxl_pt[0] = np.clip(pxl_pt[0], a_min=0, a_max=map_height-1)
                    if initial_region_position_matching(pxl_pt) == 'not_initial_region':
                        continue
                    if initial_region_position_matching(pxl_pt) == '1_in1':
                        initial_vehicle_set['1_in1'].append(v)
                    if initial_region_position_matching(pxl_pt) == '1_in2':
                        initial_vehicle_set['1_in2'].append(v)
                    if initial_region_position_matching(pxl_pt) == '1_in3':
                        initial_vehicle_set['1_in3'].append(v)
                    if initial_region_position_matching(pxl_pt) == '2_in1':
                        initial_vehicle_set['2_in1'].append(v)
                    if initial_region_position_matching(pxl_pt) == '2_in2':
                        initial_vehicle_set['2_in2'].append(v)
                    if initial_region_position_matching(pxl_pt) == '2_in3':
                        initial_vehicle_set['2_in3'].append(v)
                    if initial_region_position_matching(pxl_pt) == '2_in4':
                        initial_vehicle_set['2_in4'].append(v)
                    if initial_region_position_matching(pxl_pt) == '3_in1':
                        initial_vehicle_set['3_in1'].append(v)
                    if initial_region_position_matching(pxl_pt) == '3_in2':
                        initial_vehicle_set['3_in2'].append(v)
                    if initial_region_position_matching(pxl_pt) == '3_in3':
                        initial_vehicle_set['3_in3'].append(v)
        except EOFError:
            logging.info(f'TIME_BUFF{buff_name} occurs EOFError, skipped')
            continue
        logging.info(f'TIME_BUFF{buff_name} searched successfully')

with open(output_initial_vehicle_filename, 'wb') as f:
    pickle.dump(initial_vehicle_set, f)

logging.info(f'initial_vehicle_dict generated successfully')

In [8]:
import pandas as pd
import pickle
import os
from vehicle import Vehicle

initial_pickle_path = 'data/inference/AA_rdbt/simulation_initialization/gen_veh_states/AA_rdbt/initial_vehicle_dict.pickle'

with open(initial_pickle_path, 'rb') as f:
    vehicle_dict = pickle.load(f)
    
    # 现在vehicle_dict包含了initial pickle文件中每个入口的Vehicle列表
    print(vehicle_dict.keys())
    vehicle_list_n_in_1 = vehicle_dict['n_in1']
    vehicle_list_n_in_2 = vehicle_dict['n_in2']
    vehicle_list_n_in_3 = vehicle_dict['n_in3']
    vehicle_list_e_in_1 = vehicle_dict['e_in1']
    vehicle_list_e_in_2 = vehicle_dict['e_in2']
    vehicle_list_s_in_1 = vehicle_dict['s_in1']
    vehicle_list_s_in_2 = vehicle_dict['s_in2']
    vehicle_list_w_in_1 = vehicle_dict['w_in1']
    vehicle_list_w_in_2 = vehicle_dict['w_in2']
    print(f'''
        n_in_1 vehile numbers: {len(vehicle_list_n_in_1)}
        n_in_2 vehile numbers: {len(vehicle_list_n_in_2)}
        n_in_3 vehile numbers: {len(vehicle_list_n_in_3)}
        e_in_1 vehile numbers: {len(vehicle_list_e_in_1)}
        e_in_2 vehile numbers: {len(vehicle_list_e_in_2)}
        s_in_1 vehile numbers: {len(vehicle_list_s_in_1)}
        s_in_2 vehile numbers: {len(vehicle_list_s_in_2)}
        w_in_1 vehile numbers: {len(vehicle_list_w_in_1)}
        w_in_2 vehile numbers: {len(vehicle_list_w_in_2)}''')
    print(vehicle_list_n_in_1[0].location.x)
    print(vehicle_list_n_in_1[0].location.y)
    print(vehicle_list_n_in_1[0].size.width) 
    print(vehicle_list_n_in_1[0].size.height)
    print(vehicle_list_n_in_1[0].size.length)
    print(vehicle_list_n_in_1[0].id) 
    print(vehicle_list_n_in_1[0].speed_heading)
    print(vehicle_list_n_in_1[0].poly_box)
    print(vehicle_list_n_in_1[0].realworld_4_vertices)
    print(vehicle_list_n_in_1[0].safe_poly_box)

/tmp/ipykernel_555730/1959126300.py:9: UserWarning: Unpickling a shapely <2.0 geometry object. Please save the pickle again; shapely 2.1 will not have this compatibility.
  vehicle_dict = pickle.load(f)


dict_keys(['n_in1', 'n_in2', 'e_in1', 'e_in2', 's_in1', 's_in2', 'w_in1', 'w_in2', 'n_in3'])

        n_in_1 vehile numbers: 4507
        n_in_2 vehile numbers: 867
        n_in_3 vehile numbers: 1384
        e_in_1 vehile numbers: 3960
        e_in_2 vehile numbers: 3345
        s_in_1 vehile numbers: 2202
        s_in_2 vehile numbers: 2792
        w_in_1 vehile numbers: 7898
        w_in_2 vehile numbers: 1844
70.15242977004964
-7.438291377387941
1.8
1.5
3.6
7
258.27237614762805
POLYGON ((70.66777531778924 -9.38364969050791, 68.90535044819742 -9.017782803451603, 69.63708422231004 -5.492933064267971, 71.39950909190185 -5.858799951324279, 70.66777531778924 -9.38364969050791))
[[70.66777532 -9.38364969]
 [68.90535045 -9.0177828 ]
 [69.63708422 -5.49293306]
 [71.39950909 -5.85879995]]
POLYGON ((70.74536187237454 -9.501888121432806, 68.78711201727252 -9.095369358036908, 69.55949766772473 -5.374694633343076, 71.51774752282675 -5.781213396738973, 70.74536187237454 -9.501888121432806))


In [2]:
import pandas as pd
import pickle
import os
from vehicle import Vehicle

initial_pickle_path = 'data_yz/inference/rcu_252/simulation_initialization/gen_veh_states/initial_vehicle_dict_0403.pickle'

with open(initial_pickle_path, 'rb') as f:
    vehicle_dict = pickle.load(f)
    
    # 现在vehicle_dict包含了initial pickle文件中每个入口的Vehicle列表
    print(vehicle_dict.keys())
    vehicle_list_1_in_1 = vehicle_dict['1_in1']
    vehicle_list_1_in_2 = vehicle_dict['1_in2']
    vehicle_list_1_in_3 = vehicle_dict['1_in3']
    vehicle_list_2_in_1 = vehicle_dict['2_in1']
    vehicle_list_2_in_2 = vehicle_dict['2_in2']
    vehicle_list_2_in_3 = vehicle_dict['2_in3']
    vehicle_list_2_in_4 = vehicle_dict['2_in4']
    vehicle_list_3_in_1 = vehicle_dict['3_in1']
    vehicle_list_3_in_2 = vehicle_dict['3_in2']
    vehicle_list_3_in_3 = vehicle_dict['3_in3']

    print(f'''
        1_in_1 vehile numbers: {len(vehicle_list_1_in_1)}
        1_in_2 vehile numbers: {len(vehicle_list_1_in_2)}
        1_in_3 vehile numbers: {len(vehicle_list_1_in_3)}
        2_in_1 vehile numbers: {len(vehicle_list_2_in_1)}
        2_in_2 vehile numbers: {len(vehicle_list_2_in_2)}
        2_in_3 vehile numbers: {len(vehicle_list_2_in_3)}
        2_in_4 vehile numbers: {len(vehicle_list_2_in_4)}
        3_in_1 vehile numbers: {len(vehicle_list_3_in_1)}
        3_in_2 vehile numbers: {len(vehicle_list_3_in_2)}
        3_in_3 vehile numbers: {len(vehicle_list_3_in_3)}''')
    print(vehicle_list_1_in_1[0].location.x)
    print(vehicle_list_1_in_1[0].location.y)
    print(vehicle_list_1_in_1[0].size.width) 
    print(vehicle_list_1_in_1[0].size.height)
    print(vehicle_list_1_in_1[0].size.length)
    print(vehicle_list_1_in_1[0].id) 
    print(vehicle_list_1_in_1[0].speed_heading)
    print(vehicle_list_1_in_1[0].poly_box)
    print(vehicle_list_1_in_1[0].realworld_4_vertices)
    print(vehicle_list_1_in_1[0].safe_poly_box)

dict_keys(['1_in1', '1_in2', '1_in3', '2_in1', '2_in2', '2_in3', '2_in4', '3_in1', '3_in2', '3_in3'])

        1_in_1 vehile numbers: 2455
        1_in_2 vehile numbers: 6822
        1_in_3 vehile numbers: 5878
        2_in_1 vehile numbers: 5778
        2_in_2 vehile numbers: 10186
        2_in_3 vehile numbers: 9449
        2_in_4 vehile numbers: 8087
        3_in_1 vehile numbers: 0
        3_in_2 vehile numbers: 0
        3_in_3 vehile numbers: 0
420.4425847772509
-188.97533294837922
1.45
1.37
4.06
30303030303030303030303030326233
239.2103
POLYGON ((420.0262638903792 -191.0903274919075, 418.78063858834133 -190.34808925055705, 420.8589056641226 -186.86033840485095, 422.10453096616044 -187.60257664620138, 420.0262638903792 -191.0903274919075))
[[ 420.02626389 -191.09032749]
 [ 418.78063859 -190.34808925]
 [ 420.85890566 -186.8603384 ]
 [ 422.10453097 -187.60257665]]
POLYGON ((420.32904866930056 -191.11942006232573, 418.61094480442074 -190.09564317770443, 420.5561208852012 -186.831245

In [2]:
import pandas as pd
import pickle
import os
import glob
from tqdm import tqdm
import logging
import cv2
import numpy as np
from vehicle import Vehicle
from geo_engine import GeoEngine
from ROIs_yz.ROIs_252 import ROIMatcher_252
from datetime import datetime

base_map_dir = 'data_yz/inference/rcu_252/basemap/252.png'
initail_region_dir = 'data_yz/inference/rcu_252/ROIs-map/vehicle-initial-region'
initial_pickle_dir = 'data_yz/inference/rcu_252/simulation_initialization/initial_clips'
# output_initial_vehicle_filename = 'data_yz/inference/rcu_252/simulation_initialization/gen_veh_states/initial_vehicle_dict_0403.pickle'

map_width = 1896
map_height = 2091

# ROIs
ROI_map_dir = "data_yz/inference/rcu_252/ROIs-map"
circle_map_dir = os.path.join(ROI_map_dir, 'circle')
entrance_map_dir = os.path.join(ROI_map_dir, 'entrance')
exit_map_dir = os.path.join(ROI_map_dir, 'exit')
crosswalk_map_dir = None
yielding_area_map_dir = os.path.join(ROI_map_dir, 'yielding-area')
at_circle_lane_map_dir = os.path.join(ROI_map_dir, 'at-circle-lane')

drivable_map_dir = "data_yz/inference/rcu_252/drivablemap/252-drivablemap.png"
sim_remove_vehicle_area_map = "data_yz/inference/rcu_252/ROIs-map/252-sim-remove-vehicle-area-map.png"  # E.g., exits.
entrance_map_dir = "data_yz/inference/rcu_252/ROIs-map/entrance"

ROIMatcher = ROIMatcher_252(drivable_map_dir=drivable_map_dir, sim_remove_vehicle_area_map_dir=sim_remove_vehicle_area_map, circle_map_dir=circle_map_dir,
                                      entrance_map_dir=entrance_map_dir, exit_map_dir=exit_map_dir, crosswalk_map_dir=crosswalk_map_dir, yielding_area_map_dir=yielding_area_map_dir,
                                      at_circle_lane_map_dir=at_circle_lane_map_dir,
                                      map_height=map_height, map_width=map_width)
    
base_map = GeoEngine(base_map_dir, map_height, map_width)

time_format = '%Y-%m-%d %H-%M-%S-%f'
start_time = '2024-01-24 08-24-31-991000'
end_time = '2024-01-24 12-26-01-828000'
time_region = datetime.strptime(end_time, time_format) - datetime.strptime(start_time, time_format)
time_region_h = time_region.total_seconds() / 3600

logging.basicConfig(filename='252_arr_rate_cal_0408.log', level=logging.INFO)

path_to_traj_data = initial_pickle_dir
subfolders = sorted(os.listdir(os.path.join(path_to_traj_data))) # 001
subsubfolders = [sorted(os.listdir(os.path.join(path_to_traj_data, subfolders[i]))) for i in
                    range(len(subfolders))] # 以rcu_252_0306为例，002~007
# print(len(subfolders))
# print(subfolders)
# print(len(subsubfolders[0])) # 
# print(subsubfolders[0][1]) #

num_1_in1, num_1_in2, num_1_in3 = 0, 0, 0
num_2_in1, num_2_in2, num_2_in3, num_2_in4 = 0, 0, 0, 0
uid_list = [] # 统计车流量，uid相同的车在同一个入口内只算一次，仿真中初始化取了一个车长内的车辆，所以可以认为初始化字典中不会重复

for i in range(len(subfolders)):
    for j in tqdm(range(len(subsubfolders[i])), desc='counting veh/h'): # len(subsubfolders[i])
        # print(j)
        files_list = sorted(glob.glob(os.path.join(path_to_traj_data, subfolders[i], subsubfolders[i][j], '*.pickle')))
        buff_name = subsubfolders[i][j]
        # print(f'start generating features of {buff_name}')
        pickle_sum = len(files_list)
        try:
            for k in tqdm(range(pickle_sum),desc='counting vehicles of subsubfolder entrance'):
                vehicle_list = pickle.load(open(files_list[k], "rb")) # file_list[i]是一个pickle，即一帧
                for v in vehicle_list:
                    if v.id in uid_list: # 一开始不在入口，按时序也不会再出现在入口
                        continue
                    else:
                        uid_list.append(v.id)
                        pxl_pt = base_map._world2pxl([v.location.x, v.location.y])
                        pxl_pt[1] = np.clip(pxl_pt[1], a_min=0, a_max=map_width-1) # road_map (map_width, map_height)
                        pxl_pt[0] = np.clip(pxl_pt[0], a_min=0, a_max=map_height-1)
                        if ROIMatcher.entrance_lane_matching(pxl_pt) == 'not_at_entrance':
                            continue
                        elif ROIMatcher.entrance_lane_matching(pxl_pt) == 'entrance_1_t_1':
                            num_1_in1 += 1
                        elif ROIMatcher.entrance_lane_matching(pxl_pt) == 'entrance_1_t_2':
                            num_1_in2 += 1
                        elif ROIMatcher.entrance_lane_matching(pxl_pt) == 'entrance_1_t_3':
                            num_1_in3 += 1
                        elif ROIMatcher.entrance_lane_matching(pxl_pt) == 'entrance_2_t_1':
                            num_2_in2 += 1
                        elif ROIMatcher.entrance_lane_matching(pxl_pt) == 'entrance_2_t_2':
                            num_2_in3 += 1
                        elif ROIMatcher.entrance_lane_matching(pxl_pt) == 'entrance_2_t_3':
                            num_2_in4 += 1
                        elif ROIMatcher.entrance_lane_matching(pxl_pt) == 'entrance_2_t_rightturn':
                            num_2_in1 += 1
                    
        except EOFError:
            logging.info(f'TIME_BUFF{buff_name} occurs EOFError, skipped')
            continue
        logging.info(f'TIME_BUFF{buff_name} calculated successfully')

print(f'arr_rate_1_in1: {round(num_1_in1/time_region_h)}, arr_rate_1_in2: {round(num_1_in2/time_region_h)}, arr_rate_1_in3: {round(num_1_in3/time_region_h)}')
print(f'arr_rate_2_in1: {round(num_2_in1/time_region_h)}, arr_rate_2_in2: {round(num_2_in2/time_region_h)}, arr_rate_2_in3: {round(num_2_in3/time_region_h)}, arr_rate_2_in4: {round(num_2_in4/time_region_h)}')

logging.info(f'num_1_in1: {num_1_in1}, num_1_in2: {num_1_in2}, num_1_in3: {num_1_in3}')
logging.info(f'num_2_in1: {num_2_in1}, num_2_in2: {num_2_in2}, num_2_in3: {num_2_in3}, num_2_in4: {num_2_in4}')
logging.info(f'arr_rate_1_in1: {round(num_1_in1/time_region_h)}, arr_rate_1_in2: {round(num_1_in2/time_region_h)}, arr_rate_1_in3: {round(num_1_in3/time_region_h)}')
logging.info(f'arr_rate_2_in1: {round(num_2_in1/time_region_h)}, arr_rate_2_in2: {round(num_2_in2/time_region_h)}, arr_rate_2_in3: {round(num_2_in3/time_region_h)}, arr_rate_2_in4: {round(num_2_in4/time_region_h)}')

logging.info(f'arrive rate calculated successfully')

counting veh/h: 100%|██████████| 109/109 [18:30<00:00, 10.19s/it]

arr_rate_1_in1: 275, arr_rate_1_in2: 345, arr_rate_1_in3: 306
arr_rate_2_in1: 461, arr_rate_2_in2: 244, arr_rate_2_in3: 282, arr_rate_2_in4: 102


In [6]:
# 示例字符串
string = "Dataset/Learning-Naturalistic-Driving-Environment/data_yz/training/252/network_training/1820/001/002/rcu_252_2024-01-24 09:51:48.908000.pickle"

# 时间戳的起始和结束索引

# 提取时间戳
timestamp = string[-33:-7]

print(timestamp)  # 输出提取的时间戳


2024-01-24 09:51:48.908000


In [8]:
# PET statistics calibration
import os
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

real_ground_truth_dir_252 = "/Dataset/Learning-Naturalistic-Driving-Environment/data_yz/statistical-realism-ground-truth/rcu_252_ground_truth/252_0409_new_PET"

gt_folder = real_ground_truth_dir_252
gt_PET_list = []

try:
    with open(os.path.join(gt_folder, "output_PET_list.json")) as json_file:
        gt_PET_list_list = json.load(json_file)
except:
    raise ValueError("No file: {0}".format(os.path.join(gt_folder, "output_PET_list.json")))

PET_threshold=30
near_miss_threshold = 0.5

# print(len(gt_PET_list_list))

for sub_list in gt_PET_list_list:
    gt_PET_list += [val for val in sub_list]

print(len(gt_PET_list))

gt_PET_list = [val for val in gt_PET_list if val < PET_threshold]
gt_near_miss_num = [val for val in gt_PET_list if val < near_miss_threshold]

print(f'when near_miss_threshold = {near_miss_threshold}, there is {len(gt_near_miss_num)} near miss events in {len(gt_PET_list)} total PETs, constitues{float(len(gt_near_miss_num))/float(len(gt_PET_list))*100}%')

near_miss_threshold = 1.0

gt_PET_list = [val for val in gt_PET_list if val < PET_threshold]
gt_near_miss_num = [val for val in gt_PET_list if val < near_miss_threshold]

print(f'when near_miss_threshold = {near_miss_threshold}, there is {len(gt_near_miss_num)} near miss events in {len(gt_PET_list)} total PETs, constitues{float(len(gt_near_miss_num))/float(len(gt_PET_list))*100}%')


1472265
when near_miss_threshold = 0.5, there is 20568 near miss events in 1328313 total PETs, constitues1.54843022691188%
when near_miss_threshold = 1.0, there is 40543 near miss events in 1328313 total PETs, constitues3.052217361420087%
